In [ ]:
!pip install wandb

In [ ]:
pip install neuralprophet

## 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from neuralprophet import NeuralProphet

In [ ]:
import wandb
from wandb.lightgbm import wandb_callback

## 시간 변환
데이터 시간을 1-24에서 0-23으로 변경

In [ ]:
def convert_time(x) :
    Ymd, HMS = x.split(' ')
    H, M, S = HMS.split(':')
    H = str(int(H)-1)
    HMS = ':'.join([H, M, S])
    return ' '.join([Ymd, HMS])

## 데이터 불러오기

In [ ]:
path = '/content/drive/MyDrive/solar_data'
# 데이터 불러오기
train_data = pd.read_csv(path + '/energy.csv')
# 시간 변환
train_data['time'] = train_data['time'].apply(lambda x : convert_time(x))
# 결측치 평균으로 상쇄
train_data['dangjin_floating'].fillna(train_data['dangjin_floating'].mean(), inplace=True)
train_data['dangjin_warehouse'].fillna(train_data['dangjin_warehouse'].mean(), inplace=True)

### 훈련 데이터 만들기 및 모델
ds = 시간, y = 데이터

In [ ]:
submission = pd.read_csv(path+'/sample_submission.csv')
def predict_col(col_list) :
    for column in col_list : 
        df = pd.DataFrame()
        df['ds'] = train_data['time']
        df['y'] = train_data[column]

        # 모델 설정
        model = NeuralProphet(seasonality_mode='multiplicative', 
                              yearly_seasonality=False, weekly_seasonality=False,
                              learning_rate=0.001, epochs=50, batch_size=128)
        # 훈련
        loss = model.fit(df, freq='H')
        # 예측용 데이터 프레임 만들기
        df_pred = model.make_future_dataframe(df, periods=18000)
        # 예측 
        predict = model.predict(df_pred)

        # 2021-02-01 ~ 2021-03-01
        predict_1 = predict.copy()
        predict_1 = predict_1.query('ds >= "2021-02-01 00:00:00"')
        predict_1 = predict_1.query('ds < "2021-03-01 00:00:00"')

        # 2021-06-09 ~ 2021-07-09
        predict_2 = predict.copy()
        predict_2 = predict_2.query('ds >= "2021-06-09 00:00:00"')
        predict_2 = predict_2.query('ds < "2021-07-09 00:00:00"')

        submission[column] = list(predict_1['yhat1']) + list(predict_2['yhat1'])
    submission.to_csv('2021_05_29.csv', index=False)

In [ ]:
col_list = ['dangjin_floating', 'dangjin_warehouse', 'dangjin', 'ulsan']
predict_col(col_list)

In [ ]:
submission

### 훈련 데이터 만들기
ds = 시간, y = 데이터

In [ ]:
path = '/content/drive/MyDrive/solar_data'
# 데이터 불러오기
train_data = pd.read_csv(path + '/energy.csv')
# 시간 변환
train_data['time'] = train_data['time'].apply(lambda x : convert_time(x))
# 결측치 평균으로 상쇄
train_data['dangjin_floating'].fillna(train_data['dangjin_floating'].mean(), inplace=True)
train_data['dangjin_warehouse'].fillna(train_data['dangjin_warehouse'].mean(), inplace=True)

In [ ]:
column = 'dangjin_floating'
df = pd.DataFrame()
df['ds'] = train_data['time']
df['y'] = train_data[column]

### 훈련 및 예측

In [ ]:
# 모델 설정
model = NeuralProphet(seasonality_mode='multiplicative',
                      yearly_seasonality=False, weekly_seasonality=False,)
# 훈련
loss = model.fit(df, freq='H')
# 예측용 데이터 프레임 만들기
df_pred = model.make_future_dataframe(df, periods=18000)
# 예측 
predict = model.predict(df_pred)

In [ ]:
model.plot(predict)

In [ ]:
model.plot_components(predict)

In [ ]:
model.plot_parameters()

In [ ]:
submission = pd.read_csv(path+'/sample_submission.csv')
# 2021-02-01 ~ 2021-03-01
predict_1 = predict_1.query('ds >= "2021-02-01 00:00:00"')
predict_1 = predict_1.query('ds < "2021-03-01 00:00:00"')

# 2021-06-09 ~ 2021-07-09
predict_2 = predict.copy()
predict_2 = predict_2.query('ds >= "2021-06-09 00:00:00"')
predict_2 = predict_2.query('ds < "2021-07-09 00:00:00"')

submission[column] = list(predict_1['yhat1']) + list(predict_2['yhat1'])